In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load the dataset
file_path = 'mergeFinalC_with_local.csv'
data = pd.read_csv(file_path)

# Define PPS calculation function
def calculate_pps(row):
    if row['shot_made_flag'] == 1:
        if row['shot_distance'] > 23.75:  # Assuming 3-point shots are beyond 23.75 feet
            return 3
        else:
            return 2
    else:
        return 0

# Apply PPS calculation
data['pps'] = data.apply(calculate_pps, axis=1)

# Define zones based on shot_distance and x, y coordinates
def categorize_zone_xy(row):
    if row['shot_distance'] <= 5:
        return 'Restricted Area'
    elif row['shot_distance'] <= 15:
        if row['x'] > 0:
            return 'Right Paint (Non-RA)'
        else:
            return 'Left Paint (Non-RA)'
    elif row['shot_distance'] <= 22:
        if row['x'] > 0:
            return 'Right Mid-Range'
        else:
            return 'Left Mid-Range'
    elif row['shot_distance'] > 22:
        if abs(row['x']) > 22:
            if row['x'] > 0:
                return 'Right Corner 3'
            else:
                return 'Left Corner 3'
        else:
            if row['x'] > 0:
                return 'Right Above the Break 3'
            else:
                return 'Left Above the Break 3'
    else:
        return 'Other'

data['shot_zone_xy'] = data.apply(categorize_zone_xy, axis=1)

# Group by player and custom shot zone with x, y coordinates
grouped_xy = data.groupby(['name', 'shot_zone_xy'])

# Calculate player PPS for each zone
player_pps_xy = grouped_xy['pps'].mean().reset_index()

# Calculate league average PPS for each zone
league_grouped_xy = data.groupby(['shot_zone_xy'])
league_pps_xy = league_grouped_xy['pps'].mean().reset_index()
league_pps_xy.rename(columns={'pps': 'league_pps'}, inplace=True)

# Merge player PPS with league PPS
merged_pps_xy = pd.merge(player_pps_xy, league_pps_xy, on='shot_zone_xy')

# Calculate shot frequency for each player in each zone
player_shot_counts_xy = grouped_xy['pps'].count().reset_index()
player_shot_counts_xy.rename(columns={'pps': 'shot_count'}, inplace=True)

# Calculate total shots for each player
total_shots_xy = data.groupby('name')['pps'].count().reset_index()
total_shots_xy.rename(columns={'pps': 'total_shots'}, inplace=True)

# Merge shot counts with total shots
player_shot_counts_xy = pd.merge(player_shot_counts_xy, total_shots_xy, on='name')
player_shot_counts_xy['shot_frequency'] = player_shot_counts_xy['shot_count'] / player_shot_counts_xy['total_shots']

# Merge shot frequency data with PPS data
merged_pps_xy = pd.merge(merged_pps_xy, player_shot_counts_xy[['name', 'shot_zone_xy', 'shot_frequency']], on=['name', 'shot_zone_xy'])

# Generate recommendations
def generate_recommendation_xy(row):
    if row['pps'] > row['league_pps']:
        if row['shot_frequency'] < 0.2:  # Arbitrary threshold to avoid recommending high-frequency zones
            return 'Shoot More'
        else:
            return 'Maintain Frequency'
    else:
        return 'Shoot Less'

merged_pps_xy['recommendation'] = merged_pps_xy.apply(generate_recommendation_xy, axis=1)

# Filter to keep only the strongest recommendations (minimum 2, maximum 4 per player)
def get_top_recommendations(df):
    df = df.sort_values(by=['pps'], ascending=False)
    top_recommendations = df.head(4) if len(df) > 4 else df
    return top_recommendations

top_recommendations = merged_pps_xy.groupby('name').apply(get_top_recommendations).reset_index(drop=True)

# Format the recommendations into a single string per player
def format_recommendations(df):
    recommendations = df['shot_zone_xy'] + ": " + df['recommendation']
    return "; ".join(recommendations)

final_recommendations = top_recommendations.groupby('name').apply(format_recommendations).reset_index()
final_recommendations.columns = ['player', 'recommendations']

# Save the final recommendations to a CSV file
final_recommendations.to_csv('Player_Recommendations.csv', index=False)

# Display the final recommendations
final_recommendations.head()

C:\Users\juani\AppData\Local\Temp\ipykernel_30336\1728903199.py:97: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_recommendations = merged_pps_xy.groupby('name').apply(get_top_recommendations).reset_index(drop=True)
C:\Users\juani\AppData\Local\Temp\ipykernel_30336\1728903199.py:104: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_recommendations = top_recommendations.groupby('name').apply(format_re

,player,recommendations
0,Carlos Boozer,Restricted Area: Maintain Frequency; Left Pain...
1,Dwight Buycks,Left Corner 3: Shoot More; Left Paint (Non-RA)...
2,Ed Davis,Restricted Area: Maintain Frequency; Right Pai...
3,Jabari Brown,Right Corner 3: Shoot More; Left Above the Bre...
4,Jeremy Lin,Right Corner 3: Shoot More; Restricted Area: S...


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.patches import Circle, Rectangle, Arc

def draw_lakers_court(ax=None, lw=2, outer_lines=False):
    if ax is None:
        ax = plt.gca()

    violet = '#552583'
    yellow = '#FDB927'

    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=violet, fill=False)
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=violet)
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=violet, fill=False)
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=violet, fill=False)
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180, linewidth=lw, color=violet)
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0, linewidth=lw, color=violet, linestyle='dashed')
    restricted = Circle((0, 0), radius=30, linewidth=lw, color=violet, linestyle='dashed', fill=False)
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw, color=violet)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=violet)
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw, color=violet)
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0, linewidth=lw, color=yellow)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0, linewidth=lw, color=yellow)
    
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw, bottom_free_throw,
                      restricted, corner_three_a, corner_three_b, three_arc, center_outer_arc, center_inner_arc]
    for element in court_elements:
        ax.add_patch(element)

    if outer_lines:
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw, color=violet, fill=False)
        ax.add_patch(outer_lines)

    ax.set_xlim(-250, 250)
    ax.set_ylim(-47.5, 422.5)
    ax.set_aspect('equal')
    return ax

# Load data
data_path = 'mergeFinalC_with_local.csv'
shots_data = pd.read_csv(data_path)

# Get unique player names
players = shots_data['name'].unique()

# Create a shot chart for each player
for player in players:
    player_shots = shots_data[shots_data['name'] == player]
    
    fig, ax = plt.subplots(figsize=(15, 13))
    draw_lakers_court(ax, outer_lines=True)
    
    sns.scatterplot(data=player_shots, x='x', y='y', hue='shot_made_flag', palette=['red', 'green'], style='shot_made_flag',
                    markers=['o', 'o'], s=50, alpha=0.6)
    
    plt.xlim(-250, 250)
    plt.ylim(-47.5, 422.5)
    plt.gca().invert_yaxis()
    plt.title(f'{player} Lakers Shot Chart', fontsize=16)
    plt.legend(title='Shot Result', loc='upper right', fontsize=12, title_fontsize=12)
    
    # Save the figure
    plt.savefig(f'{player}_shot_chart.png')
    plt.close(fig)

In [16]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install requests beautifulsoup4 pillow selenium rembg


In [21]:
import os
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from rembg import remove
import time

# Function to download and save player images
def download_player_image(player_name, save_path):
    search_query = f"Los angeles Lakers {player_name} .png"
    search_url = f"https://www.google.com/search?tbm=isch&q={search_query}"
    
    # Initialize Selenium WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.get(search_url)
    
    # Wait for images to load
    time.sleep(2)
    
    # Get image results
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img')
    image_url = None
    for image_element in image_elements:
        src = image_element.get_attribute('src')
        alt = image_element.get_attribute('alt')
        if src and src.startswith('http') and 'lakers' in alt.lower():
            image_url = src
            break
    
    driver.quit()
    
    if image_url:
        image_response = requests.get(image_url)
        image = Image.open(BytesIO(image_response.content))

        # Remove the background
        image = remove(image)

        # Convert to PNG and save the image
        image = image.convert("RGBA")
        image_save_path = os.path.join(save_path, f"{player_name}.png")
        image.save(image_save_path, format="PNG")
        print(f"Saved image for {player_name}")
    else:
        print(f"No suitable image found for {player_name}")

# Function to create directory if it doesn't exist
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Main function to scrape images for all players in the dataset
def main():
    # Load player names from the dataset
    data_path = 'mergeFinalC_with_local.csv'
    shots_data = pd.read_csv(data_path)
    players = shots_data['name'].unique()

    # Create a directory to save player images
    save_path = 'player_images'
    create_directory(save_path)

    # Download and save images for each player
    for player in players:
        download_player_image(player, save_path)

if __name__ == "__main__":
    main()


No suitable image found for Carlos Boozer
Saved image for Jordan Hill
Saved image for Wesley Johnson
Saved image for Kobe Bryant
Saved image for Jeremy Lin
Saved image for Jordan Clarkson
Saved image for Ronnie Price
Saved image for Julius Randle
Saved image for Xavier Henry
Saved image for Ed Davis
Saved image for Robert Sacre
Saved image for Wayne Ellington
Saved image for Ryan Kelly
Saved image for Nick Young
Saved image for Tarik Black
Saved image for Jabari Brown
No suitable image found for Dwight Buycks
Saved image for Vander Blue


In [27]:
pip install scikit-learn


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.0 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.4/11.0 MB 4.7 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/11.0 MB 6.6 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/11.0 MB 8.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/11.0 MB 9.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/11.0 MB 9.6 MB/s eta 0:00:01
   ----------- ---------------------------- 3.3/11.0 MB 10.4 MB/s eta 0:00:01
   ------------- -------------------------- 3.8/11.0 MB 10.6 MB/s eta 0:00:01
   ---------------- ----------------------- 4.4/11.0 MB 10.9 MB/s eta 0:00:01
   ------------------ --------------------- 5.1/11.0 MB 11.2 MB/s eta 0:00:01
   -------------------- ------------------- 5.7/11.0 MB 11.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.2/11.0 MB 11.3 MB/s eta 0:00:01

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.4.2 which is incompatible.


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
data_path = 'mergeFinalC_with_local.csv'
data = pd.read_csv(data_path)

# Select relevant features for modeling
features = ['period', 'minutes_remaining', 'seconds_remaining', 'shot_distance', 'x', 'y', 'defender_distance', 'shot_clock']
target = 'shot_made_flag'

# Drop rows with missing values in relevant columns
data = data.dropna(subset=features + [target])

# Split the data into training and testing sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000)
}

# Train and evaluate models
model_accuracies = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    model_accuracies[name] = accuracy
    print(f"{name} Accuracy: {accuracy:.4f}")

# Save model accuracies to a CSV file
model_accuracies_df = pd.DataFrame(model_accuracies.items(), columns=['Model', 'Accuracy'])
model_accuracies_df.to_csv('model_accuracies.csv', index=False)

# Select the best model (assuming RandomForest for this example)
best_model = models['RandomForest']

# Compute feature importance
importances = best_model.feature_importances_
feature_importance = pd.DataFrame({'feature': features, 'importance': importances}).sort_values(by='importance', ascending=False)

print("Feature Importance:")
print(feature_importance)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Feature Importance')
plt.savefig('feature_importance.png')
plt.close()

# Separate data into 2-pointers and 3-pointers
data_2pointers = data[data['shot_distance'] <= 23.75]
data_3pointers = data[data['shot_distance'] > 23.75]

# Function to train model and get feature importance
def analyze_feature_importance(data_subset):
    X = data_subset[features]
    y = data_subset[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    best_model.fit(X_train_scaled, y_train)
    importances = best_model.feature_importances_
    feature_importance = pd.DataFrame({'feature': features, 'importance': importances}).sort_values(by='importance', ascending=False)
    return feature_importance

# Analyze feature importance for 2-pointers
feature_importance_2pointers = analyze_feature_importance(data_2pointers)
print("Feature Importance for 2-pointers:")
print(feature_importance_2pointers)

# Save feature importance for 2-pointers to a CSV file
feature_importance_2pointers.to_csv('feature_importance_2pointers.csv', index=False)

# Analyze feature importance for 3-pointers
feature_importance_3pointers = analyze_feature_importance(data_3pointers)
print("Feature Importance for 3-pointers:")
print(feature_importance_3pointers)

# Save feature importance for 3-pointers to a CSV file
feature_importance_3pointers.to_csv('feature_importance_3pointers.csv', index=False)

# Plot feature importance for 2-pointers
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_2pointers)
plt.title('Feature Importance for 2-pointers')
plt.savefig('feature_importance_2pointers.png')
plt.close()

# Plot feature importance for 3-pointers
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_3pointers)
plt.title('Feature Importance for 3-pointers')
plt.savefig('feature_importance_3pointers.png')
plt.close()

RandomForest Accuracy: 0.6168
LogisticRegression Accuracy: 0.6102
Feature Importance:
             feature  importance
5                  y    0.164983
4                  x    0.152785
6  defender_distance    0.142986
7         shot_clock    0.136384
2  seconds_remaining    0.132641
3      shot_distance    0.125795
1  minutes_remaining    0.090240
0             period    0.054186
Feature Importance for 2-pointers:
             feature  importance
5                  y    0.163193
4                  x    0.151216
6  defender_distance    0.141536
7         shot_clock    0.139342
2  seconds_remaining    0.136612
3      shot_distance    0.122180
1  minutes_remaining    0.091162
0             period    0.054759
Feature Importance for 3-pointers:
             feature  importance
4                  x    0.163812
5                  y    0.163323
6  defender_distance    0.160423
7         shot_clock    0.149256
2  seconds_remaining    0.146300
1  minutes_remaining    0.101813
3      shot_distanc